In [21]:
import json
import os
import urllib.parse
import requests

import openai
from dotenv import load_dotenv

# METAR

In [22]:
def process_metars(raw_metars, properties):
    stations = {}
    for weather_report in raw_metars:
        station = {}

        station_id = weather_report["properties"].get("id")
        if station_id not in stations:        
            stations[station_id] = station
        
        for property,feat_name in properties.items():
            station[feat_name] = weather_report["properties"].get(property, None)
        
        station["geometry"] = weather_report.get("geometry", None)
        
        stations[station_id].update(station)
    return stations

def download_metars(args, properties, base_url="https://aviationweather.gov/api/data/metar"):
    params = urllib.parse.urlencode(args)
    url = f"{base_url}?{params}"
    print("Downloading new METARS data from", url)

    resp = requests.get(url)
    if resp.status_code != 200:
        print("Error downloading METARS", resp.status_code)
        return None
    metars_json = json.loads(resp.text)
    metars = process_metars(metars_json["features"], properties)
    return metars

In [ ]:
station_ids = ["CYYZ", "OMDB", "LTFM", "KLAX", "WIII"]
weather_params = {
    "site": "station_name", 
    "obsTime": "time_utc", 
    "temp": "temperature_c",
    "dewp": "dew_point_c",
    "wspd": "wind_speed_kt", 
    "wdir": "wind_direction", 
    "wgst": "wind_gust_kt", 
    "cover": "cloud_cover",
    "wx": "wx_code"
}
args = {
    "ids": ",".join(station_ids),
    "format": "geojson",
    "taf": False,
    "hours": 1
}
metars = download_metars(args, weather_params)

In [24]:
metars

{'CYYZ': {'station_name': 'Toronto/Pearson Intl, ON, CA',
  'time_utc': '2025-02-07T18:00:00Z',
  'temperature_c': -4,
  'dew_point_c': -11,
  'wind_speed_kt': 19,
  'wind_direction': 270,
  'wind_gust_kt': None,
  'cloud_cover': 'SCT',
  'wx_code': None,
  'geometry': {'type': 'Point', 'coordinates': [-79.629, 43.679]}},
 'OMDB': {'station_name': 'Dubai Intl, DU, AE',
  'time_utc': '2025-02-07T17:30:00Z',
  'temperature_c': 24,
  'dew_point_c': 16,
  'wind_speed_kt': 8,
  'wind_direction': 100,
  'wind_gust_kt': None,
  'cloud_cover': 'CAVOK',
  'wx_code': None,
  'geometry': {'type': 'Point', 'coordinates': [55.366, 25.254]}},
 'LTFM': {'station_name': 'Istanbul Arpt, IS, TR',
  'time_utc': '2025-02-07T17:20:00Z',
  'temperature_c': 3,
  'dew_point_c': 1,
  'wind_speed_kt': 22,
  'wind_direction': 360,
  'wind_gust_kt': None,
  'cloud_cover': 'BKN',
  'wx_code': '-SHRA',
  'geometry': {'type': 'Point', 'coordinates': [28.74, 41.262]}},
 'KLAX': {'station_name': 'Los Angeles Intl, CA,

# OpenAI

In [25]:
def list_models(client):
    for m in client.models.list():
        print(m.id)

def gpt_weather_rpt(client: openai.OpenAI, model: str, messages: list, max_tokens: int):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
        max_tokens=max_tokens
    )
    weather_report = ""
    if response.choices[0].finish_reason == "stop":
        weather_report = response.choices[0].message.content
    return weather_report

In [26]:
load_dotenv() # OpenAI API Key is stored in an environment variable
openai_client = openai.OpenAI()

In [27]:
list_models(openai_client)

gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
o1-mini-2024-09-12
o1-preview-2024-09-12
o1-mini
o1-preview
chatgpt-4o-latest
whisper-1
dall-e-3
gpt-4-turbo
gpt-4-turbo-preview
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
babbage-002
omni-moderation-latest
omni-moderation-2024-09-26
tts-1-hd-1106
gpt-4o-2024-08-06
gpt-4o
gpt-4o-2024-05-13
tts-1-hd
gpt-4o-2024-11-20
gpt-4-turbo-2024-04-09
tts-1
gpt-3.5-turbo-16k
tts-1-1106
gpt-4o-mini-2024-07-18
gpt-4o-mini
davinci-002
gpt-3.5-turbo-1106
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-4o-realtime-preview-2024-10-01
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0125
gpt-4o-audio-preview-2024-12-17
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
text-embedding-3-large
gpt-4o-realtime-preview
text-embedding-3-small
gpt-4-0125-preview
gpt-4
text-embedding-ada-002
gpt-4-1106-preview
gpt-4-0613


In [28]:
INSTRUCTIONS = (
    "You are an AI assistant specialized in interpreting and describing weather data stored in JSON format. "
    "You are expert at reading weather data for a given location and providing an English description of the weather at that location. "
    "The English weather description should be in the style of TV weatherman."
)
PROMPT = (
    "Briefly describe the weather for the location identified in the following JSON data. "
    "Format your response as a JSON object with the following key: 'weather_description'. "
)

In [31]:
for station_id, weather in metars.items():
    msgs = [{"role": "system", "content": INSTRUCTIONS}]
    content = [{"type": "text", "text": f"{PROMPT} \n\nWEATHER DATA: {json.dumps(weather)}"}]
    msgs.append({"role": "user", "content": content})

    response = gpt_weather_rpt(
        client=openai_client,
        model="gpt-4o-mini",
        messages=msgs,
        max_tokens=1000
    )
    weather.update(json.loads(response))

In [35]:
with open("../data/metars/current-weather.json", "w") as f:
    json.dump(metars, f, indent=4)

In [32]:
metars

{'CYYZ': {'station_name': 'Toronto/Pearson Intl, ON, CA',
  'time_utc': '2025-02-07T18:00:00Z',
  'temperature_c': -4,
  'dew_point_c': -11,
  'wind_speed_kt': 19,
  'wind_direction': 270,
  'wind_gust_kt': None,
  'cloud_cover': 'SCT',
  'wx_code': None,
  'geometry': {'type': 'Point', 'coordinates': [-79.629, 43.679]},
  'weather_description': "Good evening, Toronto! As we take a look at the weather at Pearson International Airport, we see it's a chilly -4 degrees Celsius. With a dew point of -11, it feels even colder out there. Wind is coming from the west at 19 knots, adding a brisk chill to the air. We've got some scattered clouds dotting the sky, so it might not be completely clear tonight. Bundle up if you're heading outside!"},
 'OMDB': {'station_name': 'Dubai Intl, DU, AE',
  'time_utc': '2025-02-07T17:30:00Z',
  'temperature_c': 24,
  'dew_point_c': 16,
  'wind_speed_kt': 8,
  'wind_direction': 100,
  'wind_gust_kt': None,
  'cloud_cover': 'CAVOK',
  'wx_code': None,
  'geome